In [ ]:
### Titanic Survival Prediction

In [ ]:
##1. Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import pickle

In [ ]:
##2. Load data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [ ]:
##3. Preprocessing
def preprocess(df):
    df = df.copy()
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df.drop(['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

    le = LabelEncoder()
    df['Sex'] = le.fit_transform(df['Sex'])
    df['Embarked'] = le.fit_transform(df['Embarked'])

    return df

train_processed = preprocess(train_df)
test_processed = preprocess(test_df)

X = train_processed.drop(['Survived', 'PassengerId'], axis=1)
y = train_processed['Survived']

In [ ]:
##4. Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
##5. Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
##6. Evaluation
y_pred = clf.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

In [ ]:
##7. Save model
with open('../models/titanic_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [ ]:
##8. Predict on test data
test_passenger_ids = test_processed['PassengerId']
X_test_final = test_processed.drop(['PassengerId'], axis=1)
test_predictions = clf.predict(X_test_final)

submission = pd.DataFrame({'PassengerId': test_passenger_ids, 'Survived': test_predictions})
submission.to_csv('../data/my_submission.csv', index=False)
print("Submission file created.")